In [ ]:
## Make sure this tool Path Matches the path for YOUR computer
toolPath = r'/Users/koshnick/Box Sync/UCD_ECO_coding/MnV-Tool/src'

import sys
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
sys.path.append(toolPath)
import mnv

pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 500)

print(mnv.version)

### Local Data Loading

In [ ]:
filePath = 'data/pes kbtu.xlsx'
data = pd.read_excel(filePath, header=0, index_col=0, parse_dates=True, infer_datetime_format=True)
print(data.columns)

### PI Data Loading

In [ ]:
from PI_client import pi_client
pi = pi_client()

tags = pi.search_by_point('*PES*kbtu')[0]
# tags += pi.search_by_point('*shields*kbtu*')[0]

print(tags)

#### Pull tags

In [ ]:
startDate = '2017-01-01'
endDate = '2018-01-01'
interval = '1 hour' #Can be "minute" "hour" "day"
calculation = 'calculated' # Redundant?

# data = pi.get_stream_by_point(tags, start=startDate, end=endDate, interval=interval, calculation=calculation)

# Data Section

In [ ]:
dataParams = {'column': 0,
              'IQRmult' : 3.0,
              'IQR' : True,
              'floor': 0,
              'ceiling': 40000,
              'resampleRate' : 'D', #'D' for daily 'H' for hourly
              'OATsource' : 'file', #'self' or 'file'
              'OATname' : 'OAT', #Name of OAT column if OATsource is 'self'} #only needed with sliceType : 'ranges'
              'sliceType' : 'ranges', #half, middate, ranges
              'midDate' : '2017-01-01', #only needed with sliceType : 'middate'
              'dateRanges' : ['2016-01-01','2018-01-01','2018-01-16','2018-04-27'], #only needed with sliceType : 'ranges'
             }

dk = mnv.data_keeper(data, dataParams)
dk.default_clean()

# Plots
dk._outlier_plot()
dk._resampled_plot()
dk._pre_post_plot()

# Many Linear Models

In [ ]:
modelParams = {'var': ['CDH', 'HDH', 'month', 'hour', 'weekday'],
               'testTrainSplit': 'random',
               'randomState': 4291990,  #Good idea to use a seed value so folds are identical
               'testSize': 0.20,
               'commodityRate': 0.0651,
               'varPermuteList': ['', 'C(month)', 'C(weekday)']
              }

allmod = mnv.many_ols(dk.pre, dk.post, modelParams)
allmod.run_all_linear()

print(allmod.statsPool[0:5])  # Display top stats
allmod.plot_pool(2)  # Visualize top models

# Single Linear Model

In [ ]:
# Set the modelParam "var" to the 'best' model from run_all_linear()
modelParams['var'] = allmod.statsPool.iloc[0]['var']

mod = mnv.ols_model(dk.pre, dk.post, modelParams)
mod.calculate_kfold()
mod.calculate_vif()

print(mod.vif[mod.vif['VIF'] > 5])

print('\nKfold')
print(mod.kfoldStats)

mod.model_plot()
plt.show() # Show plot before Stats summary
mod.Fit.summary()

### Savings

In [ ]:
print("Dollar Difference = $" +str(round(mod.postDiffSum * mod.params.commodityRate, 1)))
mod.savings_plot(yaxis='dollars')

### TMY


In [ ]:
mod.generate_savings_summary()
mod.calculate_F_uncertainty()
mod.plot_tmy_comparison()

# print("TMY annual estimated difference = $" +str(round(mod.data['tmyDiff'].cumsum()[-1]*mod.params.commodityRate, 1)))

print(mod.savingsSummary)

### Creating an Archive

In [ ]:
## Only run this section when you're confident in the model and wish to save it
mnv.create_archive(dk, mod, saveFigs=True, copyRemodel=True)
